# Preparation

## Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.math as tfm
import datetime

from pandas import DataFrame
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SimpleRNN, GRU, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

## Data preparation

In [2]:
from Feature import Feature
from utils.label import label


# Dataframe with 15 minutes between each row
df_15min = label("../binance-BTCUSDT-1m.csv")


# We train the model with a half of df_15min
df = df_15min.iloc[int(len(df_15min)*0):, :]

In [3]:
a = Feature(df.copy())

a.price_EMA(days=5, inplace=True)
a.price_SMA(days=5, inplace=True)
a.price_WMA(days=5, inplace=True)
a.volumn_EMA(days=5, inplace=True)
a.volumn_SMA(days=5, inplace=True)
a.volumn_WMA(days=5, inplace=True)
a.RS(days=5, inplace=True)
a.RSI(days=5, inplace=True)
a.std(days=5, inplace=True)
a.dropna(inplace=True)
a.head(40)

,Open,High,Low,Close,Volumn,Timestamp End,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,...,label,price_EMA5,price_SMA5,price_WMA5,volumn_EMA5,volumn_SMA5,volumn_WMA5,RS5,RSI5,std5
Timestamp,,,,,,,,,,,,,,,,,,,,,
1531914840000,7417.99,7429.22,7406.23,7418.29,396.608081,1531915739999,2.942861e+06,2025,243.197524,1.804689e+06,...,1.0,7410.09,7410.090,7414.361333,380.605171,380.605171,378.430323,1.963559,66.256789,13.668992
1531915740000,7419.99,7463.11,7418.30,7450.01,643.557659,1531916639999,4.789527e+06,3626,396.967645,2.954178e+06,...,1.0,7413.322,7413.322,7417.094667,375.637412,375.637412,383.764626,1.981172,66.456147,13.219443
1531916640000,7450.00,7472.02,7441.96,7465.50,560.225183,1531917539999,4.179450e+06,3139,293.494504,2.189648e+06,...,1.0,7411.167333,7423.724,7429.324,378.949252,424.304906,473.071375,5.216052,83.912618,17.813284
1531917540000,7465.53,7467.74,7441.76,7454.01,512.810240,1531918439999,3.821516e+06,2573,247.813778,1.846443e+06,...,0.0,7412.603778,7436.426,7443.249333,376.741359,479.665026,518.378134,6.148358,86.010774,20.829161
1531918440000,7454.01,7457.25,7431.00,7446.00,409.629268,1531919339999,3.049843e+06,2501,244.163807,1.817944e+06,...,0.0,7411.646148,7441.162,7449.110667,378.213287,505.151636,529.426539,1.993704,66.596565,21.767539
1531919340000,7446.00,7450.50,7407.00,7423.67,343.474985,1531920239999,2.553904e+06,2542,180.215118,1.340348e+06,...,0.0,7412.284568,7446.762,7450.723333,377.232002,504.566086,497.58575,2.435273,70.890232,17.503079
1531920240000,7423.79,7437.84,7406.26,7415.33,439.211550,1531921139999,3.261080e+06,2693,233.356128,1.732818e+06,...,0.0,7411.858955,7447.838,7443.026,377.886192,493.939467,443.888716,1.128481,53.018137,15.347865
1531921140000,7415.34,7450.00,7386.12,7447.66,938.712476,1531922039999,6.966361e+06,4054,403.602801,2.995127e+06,...,1.0,7412.142697,7440.902,7432.19,377.450065,453.070245,425.646077,0.308719,23.589431,20.938839
1531922040000,7447.66,7469.37,7442.00,7459.95,522.462076,1531922939999,3.893901e+06,3255,307.901612,2.294947e+06,...,1.0,7411.953535,7437.334,7434.442667,377.740816,528.767704,587.526821,0.644345,39.185504,16.812910


In [4]:
a.columns

Index(['Open', 'High', 'Low', 'Close', 'Volumn', 'Timestamp End',
       'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',
       'Taker_buy_quote_asset_volume', 'Time_UTC_Start', 'label', 'price_EMA5',
       'price_SMA5', 'price_WMA5', 'volumn_EMA5', 'volumn_SMA5', 'volumn_WMA5',
       'RS5', 'RSI5', 'std5'],
      dtype='object')

,Close,std5
Timestamp,,
1531914840000,7418.29,13.668992
1531915740000,7450.01,13.219443
1531916640000,7465.50,17.813284
1531917540000,7454.01,20.829161
1531918440000,7446.00,21.767539
1531919340000,7423.67,17.503079
1531920240000,7415.33,15.347865
1531921140000,7447.66,20.938839
1531922040000,7459.95,16.812910
